In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [2]:
data = pd.read_csv("C:/Sem 4 Project/Daily_News_data/Combined_News_DJIA.csv")

In [3]:
train = data[data['Date'] < '2015-01-01']
test = data[data['Date'] > '2014-12-31']


In [4]:
len(test)

378

In [5]:
len(train)

1611

In [6]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [7]:
trainvect = CountVectorizer()

In [8]:
Trainfeature = trainvect.fit_transform(trainheadlines)

In [11]:
###Detailed view of Document Count Matrix
DTM_With_Colm = pd.DataFrame(Trainfeature.toarray(),columns= trainvect.get_feature_names())

In [12]:
Trainfeature.shape


(1611, 31675)

In [13]:
Logis = LogisticRegression()

In [14]:
Model1 = Logis.fit(Trainfeature,train['Label'])

In [15]:
Model1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [17]:
testheadlines =[]
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [18]:
len(testheadlines)

378

In [19]:
Testfeature = trainvect.transform(testheadlines)

In [20]:
Testfeature.shape

(378, 31675)

In [21]:
Predicted = Model1.predict(Testfeature)

In [22]:
Predicted.shape


(378,)

In [23]:
pd.crosstab(test["Label"], Predicted , rownames=["Actual"] , colnames= ["Predict"])


Predict,0,1
Actual,,
0,61,125
1,92,100


In [24]:
from sklearn.naive_bayes import MultinomialNB

Nb = MultinomialNB()

In [25]:
Model2 = Nb.fit(Trainfeature,train['Label'])

In [26]:
Nbpredicted = Model2.predict(Testfeature)

In [27]:
Nbpredicted.shape

(378,)

In [29]:
pd.crosstab(test["Label"], Nbpredicted, rownames= ["Acutal"],colnames=["Predict"])

Predict,0,1
Acutal,,
0,46,140
1,51,141


In [30]:
import numpy as np
from sklearn.metrics import accuracy_score
y_NaviBayes = Nbpredicted
y_true = test["Label"]
accuracy_score(y_NaviBayes,y_true)
x_Logist = Predicted
x_true = test["Label"]
accuracy_score(x_Logist,x_true)

0.42592592592592593

In [31]:
advvect = CountVectorizer(ngram_range=(1,2))
%time
advancedtrain = advvect.fit_transform(trainheadlines)
advancedtrain.shape

Wall time: 0 ns


(1611, 398396)

In [32]:
advmodel = MultinomialNB()
advancemodel = advmodel.fit(advancedtrain,train["Label"])
advancetest = advvect.transform(testheadlines)
advNBprediction = advmodel.predict(advancetest) 
advNBprediction.shape

(378,)

In [34]:
pd.crosstab(test["Label"],advNBprediction, rownames=["Acutal"],colnames=["Predicted"])

Predicted,0,1
Acutal,,
0,3,183
1,0,192


In [35]:
x_adNB = advNBprediction
x_test = test["Label"]

In [36]:
accuracy_score(x_test,x_adNB)

0.5158730158730159

In [37]:
from sklearn import metrics
metrics.accuracy_score(x_test,x_adNB)

0.5158730158730159

In [48]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

ModuleNotFoundError: No module named 'stop_words'

In [50]:
%time
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = stopwords('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
trainheadlines

# list for tokenized documents in loop
texts = []

# loop through document list
for i in trainheadlines:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

Wall time: 0 ns


TypeError: 'list' object is not callable